In [26]:
import re

In [28]:
def common_subexpression(tac):
    visited={}
    optimized=[]
    for line in tac:
        var,expr=line.split("=")
        expr=expr.strip()
        if expr in visited:
            optimized.append(f"{var}={visited[expr]}")
        else:
            visited[expr]=var
            optimized.append(line)
    return optimized

In [30]:
def copy_propagation(tac):
    replace={}
    optimized=[]
    for line in tac:
        var,expr=line.split("=")
        expr=expr.strip()
        if expr in replace:
            expr=replace[expr]
        replace[var.strip()]=expr
        optimized.append(f"{var.strip()}={expr}")
    return optimized

In [32]:
def constant_propagation(tac):
    constants={}
    optimized=[]
    for line in tac:
        var,expr=line.split("=")
        var,expr=var.strip(),expr.strip()
        if expr.isdigit():
            constants[var]=expr
        else:
            for const,value in constants.items():
                expr=re.sub(rf'\b{const}\b',value,expr)
        optimized.append(f"{var}={expr}")
    return optimized

In [34]:
def constant_folding(tac):
    optimized=[]
    for line in tac:
        var,expr=line.split("=")
        var,expr=var.strip(),expr.strip()
        arithmetic_match=re.match(r'^(\d+)\s*([+\-*/])\s*(\d+)$',expr)
        if arithmetic_match:
            val1,op,val2=arithmetic_match.groups()
            try:
                if op=='+':
                    result=int(val1)+int(val2)
                elif op=='-':
                    result=int(val1)-int(val2)
                elif op=='*':
                    result=int(val1)*int(val2)
                elif op=='/':
                    result=int(val1)//int(val2) 
                expr=str(result)
            except (ValueError,ZeroDivisionError):
                pass      
        optimized.append(f"{var}={expr}")  
    return optimized

In [36]:
def dead_code_elimination(tac):
    used_vars=set()
    for line in tac:
        var, expr=line.split("=")
        var, expr=var.strip(),expr.strip()
        for potential_var in re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*',expr):
            used_vars.add(potential_var)    
    optimized=[]
    defined_vars=set()    
    for line in tac:
        var,expr=line.split("=")
        var,expr=var.strip(),expr.strip()
        if var in used_vars or var.startswith('output'):
            optimized.append(line)
            defined_vars.add(var)
    return optimized

In [38]:
def optimize_tac(tac):
    current_lines=tac.copy()
    previous_lines=[]
    iterations=0
    print("Starting optimization process...")
    while current_lines!=previous_lines:
        iterations+=1
        previous_lines=current_lines.copy()
        print(f"\nIteration {iterations}:") 
        temp=common_subexpression(current_lines)
        if temp!=current_lines:
            print("Applied common subexpression elimination:")
            print("\n".join(temp))
        current_lines=temp     
        temp=copy_propagation(current_lines)
        if temp!=current_lines:
            print("Applied copy propagation:")
            print("\n".join(temp))
        current_lines=temp
        temp=constant_propagation(current_lines)
        if temp!=current_lines:
            print("Applied constant propagation:")
            print("\n".join(temp))
        current_lines=temp    
        temp=constant_folding(current_lines)
        if temp!=current_lines:
            print("Applied constant folding:")
            print("\n".join(temp))
        current_lines=temp     
        temp=dead_code_elimination(current_lines)
        if temp!=current_lines:
            print("Applied dead code elimination:")
            print("\n".join(temp))
        current_lines=temp
    constants={}
    for line in current_lines:
        var,expr=line.split("=")
        var,expr=var.strip(),expr.strip()
        if expr.isdigit():
            constants[var]=expr  
    for i,line in enumerate(current_lines):
        if line.startswith("output"): 
            var,expr=line.split("=")
            var,expr=var.strip(),expr.strip()
            if expr in constants:  
                print("\nDirectly propagating constant to output variable:")
                current_lines[i]=f"{var}={constants[expr]}"  
                print(f"{current_lines[i]}") 
    
    print(f"\nOptimization completed after {iterations} iterations.")
    return current_lines

In [40]:
tac_input1=["a=5","b=10","c=a+b","d=c","e=d+b","f=a+b","g=e*2","h=d*3","output=g"]
print("Original TAC:")
for line in tac_input1:
    print(line)
optimized_tac=optimize_tac(tac_input1)
print("\nOptimized Three Address Code:")
for line in optimized_tac:
    print(line)

Original TAC:
a=5
b=10
c=a+b
d=c
e=d+b
f=a+b
g=e*2
h=d*3
output=g
Starting optimization process...

Iteration 1:
Applied common subexpression elimination:
a=5
b=10
c=a+b
d=c
e=d+b
f=c
g=e*2
h=d*3
output=g
Applied copy propagation:
a=5
b=10
c=a+b
d=a+b
e=d+b
f=a+b
g=e*2
h=d*3
output=e*2
Applied constant propagation:
a=5
b=10
c=5+10
d=5+10
e=d+10
f=5+10
g=e*2
h=d*3
output=e*2
Applied constant folding:
a=5
b=10
c=15
d=15
e=d+10
f=15
g=e*2
h=d*3
output=e*2
Applied dead code elimination:
d=15
e=d+10
output=e*2

Iteration 2:
Applied constant propagation:
d=15
e=15+10
output=e*2
Applied constant folding:
d=15
e=25
output=e*2
Applied dead code elimination:
e=25
output=e*2

Iteration 3:
Applied constant propagation:
e=25
output=25*2
Applied constant folding:
e=25
output=50
Applied dead code elimination:
output=50

Iteration 4:

Optimization completed after 4 iterations.

Optimized Three Address Code:
output=50


In [42]:
tac_input2=["t1=5","t2=10","t3=t1+t2","t4=t1+t2","t5=t3","t6=4*2","t7=t6+1","t8=t7-t3","t9=100","output1=t8"]
print("Original TAC:")
for line in tac_input2:
    print(line)
optimized_tac=optimize_tac(tac_input2)
print("\nFinal Optimized TAC:")
for line in optimized_tac:
    print(line)

Original TAC:
t1=5
t2=10
t3=t1+t2
t4=t1+t2
t5=t3
t6=4*2
t7=t6+1
t8=t7-t3
t9=100
output1=t8
Starting optimization process...

Iteration 1:
Applied common subexpression elimination:
t1=5
t2=10
t3=t1+t2
t4=t3
t5=t3
t6=4*2
t7=t6+1
t8=t7-t3
t9=100
output1=t8
Applied copy propagation:
t1=5
t2=10
t3=t1+t2
t4=t1+t2
t5=t1+t2
t6=4*2
t7=t6+1
t8=t7-t3
t9=100
output1=t7-t3
Applied constant propagation:
t1=5
t2=10
t3=5+10
t4=5+10
t5=5+10
t6=4*2
t7=t6+1
t8=t7-t3
t9=100
output1=t7-t3
Applied constant folding:
t1=5
t2=10
t3=15
t4=15
t5=15
t6=8
t7=t6+1
t8=t7-t3
t9=100
output1=t7-t3
Applied dead code elimination:
t3=15
t6=8
t7=t6+1
output1=t7-t3

Iteration 2:
Applied constant propagation:
t3=15
t6=8
t7=8+1
output1=t7-15
Applied constant folding:
t3=15
t6=8
t7=9
output1=t7-15
Applied dead code elimination:
t7=9
output1=t7-15

Iteration 3:
Applied constant propagation:
t7=9
output1=9-15
Applied constant folding:
t7=9
output1=-6
Applied dead code elimination:
output1=-6

Iteration 4:

Optimization completed